<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 24: Standards Gaps and Open Problems in AV Safety

**Session 5: Standards Integration and Deployment**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/08_Advanced_Topics/notebooks/24_Standards_Gaps_Open_Problems.ipynb)

**Author:** Milin Patel

---

## Learning Objectives

- Identify limitations of current safety standards for AI/ML systems
- Understand what ISO 26262 cannot handle for autonomous systems
- Recognize SOTIF gaps and "unknown unknowns" problem
- Explore unresolved research questions in AV safety
- Learn about emerging standards and future directions
- Develop critical thinking about standards limitations

---

In [ ]:
# Setup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import seaborn as sns
from IPython.display import display, HTML

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Setup complete")

---

## 1. Introduction: Why Standards Are Incomplete

### The Challenge of AI/ML Systems

**Traditional Software** (what ISO 26262 was designed for):
- Deterministic behavior
- Formally verifiable
- Complete requirements specification possible
- Failure modes enumerable

**AI/ML Systems** (what AVs use):
- Non-deterministic (probabilistic)
- Not formally verifiable (black box)
- Complete specification impossible (unknown unknowns)
- Failure modes unbounded (long tail)

### Current Standards Were Not Designed for AI

| Standard | Original Purpose | When Published | AI Considered? |
|----------|-----------------|----------------|----------------|
| **ISO 26262** | E/E systems with deterministic SW | 2011 (2018 ed 2) | Minimal |
| **ISO 21448** | Performance limitations | 2022 | Yes, but gaps |
| **ISO/SAE 21434** | Cybersecurity | 2021 | Partially |

**Result**: Standards provide **partial guidance** but leave major gaps

### This Notebook's Goal

**Not to criticize standards** (they're incredibly valuable!)

**But to**:
1. Understand their limitations
2. Identify what's still missing
3. Recognize active research areas
4. Prepare for future standards

**Critical thinking**: Standards are tools, not complete solutions

In [ ]:
# Visualize standards coverage of AI/ML challenges
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Chart 1: Standards applicability to AI challenges
challenges = ['Random HW\nFaults', 'SW Bugs', 'Spec\nInsufficiencies', 'Unknown\nEdge Cases', 'Adversarial\nAttacks', 'Distribution\nShift']
iso_26262 = [95, 85, 20, 10, 5, 5]  # Coverage percentage
iso_21448 = [10, 15, 80, 50, 15, 60]
iso_21434 = [5, 10, 5, 5, 85, 10]

x = np.arange(len(challenges))
width = 0.25

ax1.bar(x - width, iso_26262, width, label='ISO 26262', color='blue', alpha=0.7)
ax1.bar(x, iso_21448, width, label='ISO 21448', color='green', alpha=0.7)
ax1.bar(x + width, iso_21434, width, label='ISO/SAE 21434', color='red', alpha=0.7)

ax1.set_ylabel('Coverage (%)', fontsize=11, fontweight='bold')
ax1.set_title('Standards Coverage of AV Challenges', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(challenges, fontsize=9)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_ylim(0, 100)
ax1.axhline(y=80, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='80% threshold')

# Chart 2: Gap severity
gap_areas = ['ML\nVerification', 'Unknown\nUnknowns', 'ODD\nBoundaries', 'Online\nLearning', 'Multi-Agent\nInteraction']
severity = [95, 90, 70, 85, 75]  # How severe the gap is
colors = ['darkred', 'red', 'orange', 'red', 'orange']

bars = ax2.barh(gap_areas, severity, color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Gap Severity (Higher = More Critical)', fontsize=11, fontweight='bold')
ax2.set_title('Most Critical Standards Gaps', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')
ax2.set_xlim(0, 100)

for bar, sev in zip(bars, severity):
    ax2.text(sev + 2, bar.get_y() + bar.get_height()/2, 
             f'{sev}', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️  Key Insight: Standards cover traditional failures well, AI-specific challenges poorly")
print("   - ISO 26262: Great for HW/SW bugs, weak on AI behavior")
print("   - ISO 21448: Addresses performance limits, but 'unknown unknowns' remain")
print("   - ISO/SAE 21434: Covers traditional attacks, less on AI-specific exploits")

---

## 2. ISO 26262 Gaps for AI/ML

### What ISO 26262 Assumes

1. **Requirements are Complete**: You can specify all correct behaviors
2. **Behavior is Deterministic**: Same input → same output
3. **Verification is Possible**: Can test all critical paths
4. **Failure Modes are Enumerable**: FMEA covers all failures

### Why These Assumptions Fail for AI

#### Gap 1: Non-Deterministic Behavior

**Problem**:
- Neural networks have stochastic elements (dropout, random initialization)
- Same input can produce different outputs (especially in ensembles)
- ISO 26262 requires deterministic behavior for ASIL D

**No Clear Solution**: 
- Use deterministic mode at inference? (Hurts performance)
- Accept non-determinism? (Violates standard)

#### Gap 2: Cannot Verify Neural Networks Formally

**Problem**:
- ISO 26262 requires verification that SW meets requirements
- Neural networks: ~100M parameters, non-linear, non-interpretable
- **Cannot prove** "This network will never misclassify a pedestrian"

**Current Workaround**:
- Statistical testing on large datasets
- But: No mathematical proof possible
- Standards don't specify acceptable test size

#### Gap 3: Data Quality Not Addressed

**Problem**:
- ISO 26262 focuses on code quality (MISRA-C, code reviews)
- For AI: **Data quality is more critical than code quality**
- No guidance on:
 - Dataset representativeness
 - Label quality assessment
 - Bias detection and mitigation
 - Train/val/test split strategies

**ISO 26262 says**: "Code shall be reviewed"

**Doesn't say**: "Training data shall represent ODD"

#### Gap 4: Runtime Adaptation

**Problem**:
- ISO 26262 assumes SW is frozen after validation
- Modern AVs use:
 - Over-the-air (OTA) updates
 - Online learning (continuous improvement)
 - Model fine-tuning

**Questions**:
- If model updated OTA, does it need re-certification?
- How to validate online learning (can't test beforehand)?
- What if update makes system worse?

**Standard is silent on this**

#### Gap 5: Diagnostic Coverage for AI

**ISO 26262 Requirement**:
- Must achieve specific diagnostic coverage (e.g., 99% for ASIL D)
- Detect faults before they cause harm

**For AI**:
- How to measure diagnostic coverage of a neural network?
- What's a "fault" in a neural network? (Dead neuron? Miscalibration?)
- No consensus on metrics

**Current Practice**: 
- Use uncertainty quantification as "diagnostic"
- But: No proof this achieves required coverage

In [ ]:
# ISO 26262 gaps for AI
gaps_26262 = {
    'Gap': [
        'Non-deterministic behavior',
        'Formal verification impossible',
        'Data quality not addressed',
        'Runtime adaptation (OTA, online learning)',
        'Diagnostic coverage for AI',
        'Long-tail failure modes',
        'Adversarial robustness'
    ],
    'ISO 26262 Assumption': [
        'Deterministic SW',
        'Formal methods apply',
        'Code quality = system quality',
        'SW frozen post-validation',
        'Faults enumerable',
        'FMEA covers all failures',
        'Random faults only'
    ],
    'Reality for AI': [
        'Stochastic (dropout, ensembles)',
        'Black box, 100M+ parameters',
        'Data quality >> code quality',
        'Continuous updates needed',
        'AI "faults" not well-defined',
        'Unbounded edge cases',
        'Adversarial attacks exist'
    ],
    'Current Workaround': [
        'Use deterministic inference (performance loss)',
        'Statistical testing (no proof)',
        'Ad-hoc data validation',
        'Re-validate each OTA (expensive)',
        'Uncertainty as proxy (unproven)',
        'SOTIF helps (but incomplete)',
        'Use ISO/SAE 21434 (partial)'
    ],
    'Status': [
        'Open problem',
        'Active research',
        'ISO TR 5469 emerging',
        'No consensus',
        'Open problem',
        'SOTIF addresses (partially)',
        'Emerging research'
    ]
}

df_26262 = pd.DataFrame(gaps_26262)

print("\n" + "="*140)
print("ISO 26262 GAPS FOR AI/ML SYSTEMS")
print("="*140 + "\n")
display(df_26262)

print("\n🚨 Key Insight: ISO 26262 was designed for traditional SW, not AI")
print("   - Core assumptions (determinism, formal verification) don't hold")
print("   - Current practice: Workarounds, no consensus")
print("   - Need: AI-specific safety standard (ISO TR 5469 is first step)")

---

## 3. ISO 21448 (SOTIF) Gaps

### What SOTIF Addresses Well

✅ **Performance Limitations**: Sensor range, algorithm accuracy

✅ **Known Triggering Conditions**: Rain, fog, glare

✅ **Scenario-Based Testing**: Identify hazardous scenarios

### The "Unknown Unknowns" Problem

#### SOTIF Four Quadrants

```
 Known Scenarios Unknown Scenarios
 ─────────────── ─────────────────
Safe Performance │ Known Safe │ Unknown Safe │
 │ │ (goal: expand) │
 ├────────────────┼──────────────────│
Unsafe Performance│ Known Unsafe │ Unknown Unsafe │
 │ (mitigate!) │ (THE PROBLEM!) │
```

**SOTIF Process**:
1. Identify known unsafe scenarios → Fix
2. Discover unknown unsafe → Move to known unsafe → Fix
3. Repeat until "unknown unsafe" is acceptably small

**The Gap**: **How do you know when "unknown unsafe" is small enough?**

#### Gap 1: No Quantitative Exit Criteria

**Problem**:
- SOTIF says: "Reduce unknown unsafe scenarios"
- Doesn't say: "To what level?"
- No metric for "how much is enough testing?"

**Current Practice**:
- Drive millions of miles
- Hope you've found most edge cases
- **No proof you're done**

**What's Needed**:
- Statistical confidence metric
- "With 95% confidence, residual risk < X"
- Standard doesn't provide this

#### Gap 2: Adversarial Scenarios

**SOTIF Assumption**: Scenarios occur naturally

**Reality**: Adversaries create scenarios intentionally
- Adversarial stop signs
- Intentional sensor spoofing
- Coordinated multi-vehicle attacks

**Gap**: SOTIF doesn't cover intentional triggering conditions

**Overlap with ISO/SAE 21434**, but:
- 21434 focuses on digital attacks
- Physical adversarial examples (e.g., stickers on signs) fall between standards

#### Gap 3: Multi-Agent Interactions

**Problem**:
- SOTIF focuses on single vehicle performance
- Real world: Multiple AVs + human drivers + cyclists + pedestrians
- **Emergent behaviors** from interactions

**Example Scenario**:
- AV #1 is too cautious (stops for shadows)
- AV #2 following, also stops
- Human driver behind AV #2, frustrated, overtakes dangerously
- **Crash caused by interaction, not single failure**

**Gap**: SOTIF doesn't model multi-agent systems

#### Gap 4: Continuous Deployment

**SOTIF Assumes**: Validation → Deployment → Done

**Reality**: 
- OTA updates monthly
- Fleet learning (update model based on field data)
- ODD expansion over time

**Gap**: How to apply SOTIF to continuously evolving systems?

**Questions**:
- Does each OTA update need full SOTIF process?
- How to validate fleet learning (happens post-deployment)?
- When can you expand ODD?

In [ ]:
# SOTIF gaps visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Chart 1: SOTIF quadrants with uncertainty
from matplotlib.patches import Rectangle

ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.set_aspect('equal')

# Known safe
known_safe = Rectangle((0, 5), 5, 5, fill=True, alpha=0.5, color='green', label='Known Safe')
ax1.add_patch(known_safe)

# Known unsafe
known_unsafe = Rectangle((0, 0), 5, 5, fill=True, alpha=0.5, color='red', label='Known Unsafe')
ax1.add_patch(known_unsafe)

# Unknown safe
unknown_safe = Rectangle((5, 5), 5, 5, fill=True, alpha=0.3, color='lightgreen', label='Unknown Safe')
ax1.add_patch(unknown_safe)

# Unknown unsafe - THE PROBLEM
unknown_unsafe = Rectangle((5, 0), 5, 5, fill=True, alpha=0.7, color='darkred', label='Unknown Unsafe')
ax1.add_patch(unknown_unsafe)

# Question mark
ax1.text(7.5, 2.5, '?', fontsize=100, ha='center', va='center', color='white', fontweight='bold')

# Labels
ax1.text(2.5, 7.5, 'Validated\nScenarios', ha='center', va='center', fontsize=10, fontweight='bold')
ax1.text(2.5, 2.5, 'Known\nHazards', ha='center', va='center', fontsize=10, fontweight='bold', color='white')
ax1.text(7.5, 7.5, 'Probably\nSafe', ha='center', va='center', fontsize=10, fontweight='bold')
ax1.text(7.5, 0.5, 'How big is\nthis region?', ha='center', va='bottom', fontsize=9, fontweight='bold', color='yellow')

ax1.set_xlabel('Scenario Knowledge', fontsize=11, fontweight='bold')
ax1.set_ylabel('Safety', fontsize=11, fontweight='bold')
ax1.set_title('SOTIF Quadrants: The "Unknown Unsafe" Problem', fontsize=12, fontweight='bold')
ax1.set_xticks([2.5, 7.5])
ax1.set_xticklabels(['Known', 'Unknown'])
ax1.set_yticks([2.5, 7.5])
ax1.set_yticklabels(['Unsafe', 'Safe'])

# Chart 2: Gap severity for SOTIF
sotif_gaps = ['Exit Criteria\n(When done?)', 'Adversarial\nScenarios', 'Multi-Agent\nInteractions', 
              'Continuous\nDeployment', 'Rare Event\nValidation']
severity = [90, 80, 75, 85, 95]
colors = ['darkred', 'red', 'orange', 'red', 'darkred']

bars = ax2.barh(sotif_gaps, severity, color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Gap Severity', fontsize=11, fontweight='bold')
ax2.set_title('SOTIF Critical Gaps', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')
ax2.set_xlim(0, 100)

for bar, sev in zip(bars, severity):
    ax2.text(sev + 2, bar.get_y() + bar.get_height()/2, 
             f'{sev}', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n❓ Key Insight: SOTIF's biggest gap is 'unknown unknowns'")
print("   - No way to measure size of 'unknown unsafe' region")
print("   - No exit criteria (when is validation done?)")
print("   - Cannot prove absence of rare edge cases")
print("   - This is a FUNDAMENTAL limitation, not just implementation gap")

---

## 4. Open Research Problems

### Problem 1: Formal Verification of Neural Networks

**Goal**: Prove mathematical properties of neural networks

**Example Property**: "For all inputs representing pedestrians, output confidence > 0.95"

**Current State**:
- Possible for small networks (<1000 neurons)
- Tools: Reluplex, Marabou, CROWN
- **Problem**: Doesn't scale to ImageNet-size networks (100M parameters)
- **Timeline**: 10+ years until practical

**Implication**: Cannot formally verify AV perception systems yet

### Problem 2: Measuring Dataset Representativeness

**Question**: "Is my training dataset representative of the ODD?"

**Why It Matters**:
- If dataset biased, model will fail in deployment
- Example: Trained only on sunny days → fails in rain

**Current State**:
- No quantitative metric
- Heuristics: "Collect data from different cities, times, weather"
- **Cannot prove** "This dataset covers 99% of ODD"

**Research Directions**:
- Coverage metrics for high-dimensional spaces
- Distribution shift detection
- Active learning to find gaps

### Problem 3: Safety Validation at Scale

**Kalra & Paddock (2016)**: Need 11 billion miles to prove 20% safer

**Current Approaches**:
- Simulation (CARLA, SUMO)
- Scenario-based testing
- Accelerated testing

**Gaps**:
- **Simulation fidelity**: Real world always has surprises
- **Scenario coverage**: Cannot enumerate all scenarios
- **Statistical confidence**: No accepted methodology

**Open Problem**: How to achieve statistical proof of safety without billions of real miles?

### Problem 4: Continuous Learning and Safety

**Desired**: AV improves over time (fleet learning)

**Problem**: Learning can make system worse
- Catastrophic forgetting
- Distribution shift in field data
- Adversarial data poisoning

**Question**: How to allow online learning while maintaining safety guarantees?

**Current State**:
- Most AVs don't use online learning (too risky)
- Tesla Autopilot does, but incidents suggest problems
- No safety standard for online learning

### Problem 5: Multi-Agent Safety

**Scenario**: 1000 AVs in same city

**Questions**:
- How do they interact?
- Can emergent traffic jams occur?
- What if all AVs make same mistake (e.g., avoid phantom obstacle)?
- Adversary exploits predictable AV behavior?

**Current State**:
- Game theory models exist
- But: No validation methodology
- No standard for multi-AV safety

### Problem 6: Ethical Decision-Making

**Trolley Problem for AVs**:
- Unavoidable crash
- Must choose: Hit pedestrian? Or swerve into wall (injure passenger)?

**Questions**:
- Who decides the ethical algorithm?
- Should it be standardized?
- Can passengers override? (Leads to safety issues)
- Legal liability?

**Current State**:
- Germany has ethics commission recommendations
- No international consensus
- Standards are silent on this

### Problem 7: Long-Term Reliability

**Question**: Do neural networks degrade over time?

**Concerns**:
- Hardware aging (bit flips in weights?)
- Accumulation of small errors
- Distribution shift as world changes

**Example**:
- AV trained in 2020
- In 2030: New car designs, new traffic patterns, new road markings
- Does model still work?

**Gap**: No long-term reliability models for AI systems

In [ ]:
# Open research problems matrix
problems_data = {
    'Research Problem': [
        'Formal verification of NNs',
        'Dataset representativeness',
        'Safety validation at scale',
        'Continuous learning safety',
        'Multi-agent interactions',
        'Ethical decision-making',
        'Long-term reliability'
    ],
    'Why It Matters': [
        'Cannot prove correctness',
        'Biased training → field failures',
        'Need 11B miles (impossible)',
        'Learning can degrade safety',
        'Fleet behavior unpredictable',
        'Crash liability unclear',
        'Unknown degradation over years'
    ],
    'Current State': [
        'Only small networks (<1K neurons)',
        'Heuristics only',
        'Simulation + hope',
        'Mostly avoided (too risky)',
        'Game theory, no validation',
        'Regional guidelines, no standard',
        'No models exist'
    ],
    'Severity (1-10)': [9, 8, 10, 8, 7, 6, 7],
    'Timeline to Solution': [
        '10+ years',
        '5-10 years',
        '10+ years',
        '5+ years',
        '10+ years',
        'Unknown (political)',
        '10+ years'
    ]
}

df_problems = pd.DataFrame(problems_data)

print("\n" + "="*140)
print("OPEN RESEARCH PROBLEMS IN AV SAFETY (2025)")
print("="*140 + "\n")
display(df_problems)

print("\n🔬 Key Insight: Many fundamental problems remain unsolved")
print("   - Most critical: Safety validation at scale (severity 10/10)")
print("   - Timeline: 5-10+ years for solutions")
print("   - Current practice: Workarounds, not solutions")

In [ ]:
# Visualize research problem severity and timeline
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Chart 1: Severity comparison
problems_short = ['NN\nVerification', 'Dataset\nRep.', 'Validation\nScale', 'Online\nLearning', 
                  'Multi-Agent', 'Ethics', 'Long-term\nReliability']
severity = df_problems['Severity (1-10)'].values
colors_sev = ['red' if s >= 8 else 'orange' if s >= 7 else 'yellow' for s in severity]

bars = ax1.bar(problems_short, severity, color=colors_sev, alpha=0.7, edgecolor='black')
ax1.set_ylabel('Severity (1-10)', fontsize=11, fontweight='bold')
ax1.set_title('Open Problem Severity', fontsize=12, fontweight='bold')
ax1.set_ylim(0, 11)
ax1.axhline(y=8, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Critical Threshold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3, axis='y')

for bar, sev in zip(bars, severity):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.2,
             f'{sev}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Chart 2: Timeline to solution
timeline_categories = ['< 5 years', '5-10 years', '10+ years', 'Unknown']
counts = [0, 2, 4, 1]  # Based on data
colors_time = ['green', 'yellow', 'red', 'gray']

bars = ax2.bar(timeline_categories, counts, color=colors_time, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Number of Problems', fontsize=11, fontweight='bold')
ax2.set_title('Timeline to Solutions', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

for bar, count in zip(bars, counts):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{count}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⏳ Key Insight: Most problems are 10+ years from solution")
print("   - Validation at scale: Most critical (10/10) and longest timeline")
print("   - This explains why AV deployment is slow")

---

## 5. Emerging Standards and Future Directions

### Standards in Development

#### ISO/IEC TR 5469:2024 - AI and Functional Safety

**Status**: Published (Technical Report, not full standard yet)

**Coverage**:
- Data management for AI
- Training, validation, test set requirements
- Uncertainty quantification
- Explainability (XAI)
- Monitoring and retraining

**Limitations**:
- Technical Report = guidance, not requirements
- Doesn't address formal verification gap
- No exit criteria for validation

**Timeline**: May become full standard by 2027-2028

#### ISO 34503:2023 - Test Scenarios for ADS

**Status**: Published

**Coverage**:
- ODD specification framework
- Scenario taxonomy
- Data formats for scenarios

**Limitations**:
- Describes scenarios, doesn't say how to test
- No coverage metrics

#### UL 4600 - Standard for Autonomous Products

**Publisher**: Underwriters Laboratories (US)

**Status**: Published 2020, updated 2023

**Approach**: 
- Safety case based (not prescriptive)
- "Prove your system is safe" vs "Follow these steps"

**Coverage**:
- Risk assessment
- Validation methods
- Data quality
- Runtime monitoring

**Limitation**: 
- US-centric
- Less adoption in Europe/Asia

### What's Still Missing

#### 1. Quantitative Validation Exit Criteria

**Need**: "Test X miles in Y scenarios → 95% confidence of Z safety level"

**Challenge**: No consensus on statistical methodology

#### 2. Online Learning Standard

**Need**: How to safely update models post-deployment

**Challenge**: Validation before learning is impossible

#### 3. AI Diagnostic Coverage Metrics

**Need**: Equivalent of ISO 26262 diagnostic coverage for AI

**Challenge**: What's a "fault" in a neural network?

#### 4. Multi-Agent Safety Standard

**Need**: Safety when many AVs interact

**Challenge**: Emergent behaviors hard to predict

### Industry Feedback on Standards

**Common Complaints**:
1. **Too Slow**: Standards take 5-10 years to develop; tech moves faster
2. **Too Prescriptive**: "Do X" doesn't work for AI (need outcome-based)
3. **Fragmented**: Need to comply with 26262 + 21448 + 21434 + regional regulations
4. **Gaps**: Key problems (like validation scale) not addressed

**Suggestions**:
1. **Agile Standards**: Update every 2-3 years, not 10
2. **Outcome-Based**: Define safety goals, not implementation
3. **Harmonization**: Single global standard (unlikely politically)
4. **AI-Specific**: Need ISO 26262-like standard designed for AI

In [ ]:
# Standards timeline and coverage
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Chart 1: Standards development timeline
standards_timeline = {
    'Standard': ['ISO 26262\n(Ed 1)', 'ISO 26262\n(Ed 2)', 'ISO/SAE\n21434', 'ISO 21448\n(SOTIF)', 
                 'ISO/IEC TR\n5469', 'ISO 34503', 'UL 4600', 'ISO 26262\nfor AI?'],
    'Year': [2011, 2018, 2021, 2022, 2024, 2023, 2020, 2028],
    'Status': ['Published', 'Published', 'Published', 'Published', 'Published (TR)', 'Published', 'Published', 'Proposed']
}

years = standards_timeline['Year']
y_pos = np.arange(len(standards_timeline['Standard']))
colors_timeline = ['blue', 'blue', 'red', 'green', 'purple', 'orange', 'brown', 'gray']

ax1.barh(y_pos, [2025 - y for y in years], left=years, color=colors_timeline, alpha=0.6, edgecolor='black')

for i, (year, std) in enumerate(zip(years, standards_timeline['Standard'])):
    ax1.text(year - 0.5, i, str(year), va='center', ha='right', fontsize=9, fontweight='bold')

ax1.set_yticks(y_pos)
ax1.set_yticklabels(standards_timeline['Standard'], fontsize=9)
ax1.set_xlabel('Year', fontsize=11, fontweight='bold')
ax1.set_title('AV Safety Standards Timeline', fontsize=12, fontweight='bold')
ax1.set_xlim(2010, 2030)
ax1.axvline(x=2025, color='red', linestyle='--', linewidth=2, label='Today')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='x')

# Chart 2: Standards coverage of AI challenges (updated with new standards)
ai_challenges = ['Data\nQuality', 'Online\nLearning', 'NN\nVerification', 'Multi-Agent', 'Validation\nScale']
iso_tr_5469 = [70, 40, 20, 10, 30]  # New AI standard
ul_4600 = [60, 30, 15, 25, 40]
combined = [80, 50, 25, 30, 45]  # Using all standards together

x = np.arange(len(ai_challenges))
width = 0.25

ax2.bar(x - width, iso_tr_5469, width, label='ISO/IEC TR 5469', color='purple', alpha=0.7)
ax2.bar(x, ul_4600, width, label='UL 4600', color='brown', alpha=0.7)
ax2.bar(x + width, combined, width, label='All Standards Combined', color='green', alpha=0.7)

ax2.set_ylabel('Coverage (%)', fontsize=11, fontweight='bold')
ax2.set_title('Emerging Standards Coverage of AI Challenges', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(ai_challenges, fontsize=9)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim(0, 100)
ax2.axhline(y=80, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='80% threshold')

plt.tight_layout()
plt.show()

print("\n📋 Key Insight: New standards emerging, but gaps remain")
print("   - ISO/IEC TR 5469: Best AI coverage so far (data quality, online learning)")
print("   - Even combined: <50% coverage of validation scale, multi-agent")
print("   - Need: 5-10 more years of standards development")

---

## 6. Summary and Critical Thinking

### Main Takeaways

#### 1. Standards Are Necessary But Insufficient

✅ **Standards provide**:
- Common terminology
- Process frameworks
- Best practices
- Regulatory acceptance

❌ **Standards don't provide**:
- Solutions to fundamental AI problems
- Quantitative validation methods
- Formal verification for AI
- Exit criteria for testing

**Conclusion**: Use standards, but don't assume compliance = safety

#### 2. Biggest Gaps

**Severity Ranking**:
1. **Validation at scale** (10/10) - Cannot prove safety with finite testing
2. **NN formal verification** (9/10) - Cannot prove correctness mathematically
3. **Unknown unknowns** (9/10) - Cannot bound residual risk
4. **Online learning** (8/10) - Cannot safely update post-deployment
5. **Multi-agent** (7/10) - Cannot predict fleet interactions

#### 3. Timeline to Solutions

**Realistic Assessment**:
- **Data quality standards**: 2-5 years (ISO TR 5469 progress)
- **ODD frameworks**: Mostly solved (ISO 34503)
- **Validation methodology**: 10+ years (fundamental statistics problem)
- **Formal verification**: 10+ years (scalability challenge)
- **Multi-agent safety**: 10+ years (complexity challenge)

**Implication**: Current standards are interim solutions; expect major revisions by 2030-2035

#### 4. What This Means for Practitioners

**If you're developing AVs**:
1. **Follow standards** (regulatory requirement, best practices)
2. **Go beyond standards** (they're not sufficient)
3. **Document gaps** (for safety case, explain what's not covered)
4. **Stay conservative** (narrow ODD, extensive testing)
5. **Monitor research** (solutions emerging, be ready to adopt)

**If you're a regulator**:
1. **Standards are baseline** (not complete safety proof)
2. **Demand transparency** (what's NOT covered?)
3. **Require safety cases** (argumentation, not just compliance)
4. **Allow iteration** (tech evolving, regulations must too)

**If you're a researcher**:
1. **Work on gaps** (huge opportunity, societal impact)
2. **Collaborate with industry** (practical constraints matter)
3. **Engage with standards bodies** (influence future standards)

### The Path Forward

#### Short Term (2025-2030)
- **ISO/IEC TR 5469** becomes full standard
- Better data quality practices
- Improved scenario-based testing
- Harmonization of regional regulations

#### Medium Term (2030-2035)
- AI-specific safety standard ("ISO 26262 for AI")
- Quantitative validation methods
- Online learning guidelines
- Multi-agent safety frameworks

#### Long Term (2035+)
- Formal verification tools for large NNs
- Statistical proof of safety
- Global harmonized standard
- Level 5 enablement (maybe)

### Final Thoughts

**Standards reflect our current best understanding** — they're iterative, not perfect.

**The gaps are not failures** — they're honest acknowledgment of hard problems.

**Safety is an ongoing process** — not a one-time certification.

**Critical thinking required** — Don't blindly follow standards; understand their limitations.

**The field is young** — AV safety engineering is only ~15 years old. Traditional automotive took 100+ years to mature.

**We're making progress** — Slowly, but standards, research, and practice are improving.

**Patience needed** — This is a marathon, not a sprint. Safety cannot be rushed.

In [ ]:
# Final comprehensive summary
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 2, hspace=0.4, wspace=0.3)

# Chart 1: Standards maturity by topic
ax1 = fig.add_subplot(gs[0, :])
topics = ['HW Faults', 'SW Bugs', 'Cybersecurity', 'ODD Spec', 'Data Quality', 
          'Scenario Testing', 'NN Verification', 'Validation Scale', 'Online Learning', 'Multi-Agent']
maturity = [95, 90, 75, 80, 60, 65, 15, 25, 30, 20]  # Out of 100
colors_mat = ['green' if m >= 70 else 'yellow' if m >= 50 else 'orange' if m >= 30 else 'red' for m in maturity]

bars = ax1.barh(topics, maturity, color=colors_mat, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Standards Maturity (%)', fontsize=11, fontweight='bold')
ax1.set_title('Standards Maturity by Topic (2025)', fontsize=13, fontweight='bold')
ax1.set_xlim(0, 100)
ax1.axvline(x=70, color='green', linestyle='--', linewidth=1, alpha=0.5, label='Mature')
ax1.axvline(x=50, color='yellow', linestyle='--', linewidth=1, alpha=0.5, label='Developing')
ax1.axvline(x=30, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='Immature')
ax1.legend(loc='lower right', fontsize=9)
ax1.grid(True, alpha=0.3, axis='x')

for bar, mat in zip(bars, maturity):
    ax1.text(mat + 2, bar.get_y() + bar.get_height()/2, 
             f'{mat}%', va='center', fontsize=9, fontweight='bold')

# Chart 2: Gap severity vs timeline
ax2 = fig.add_subplot(gs[1, 0])
gap_names = ['Validation\nScale', 'NN\nVerif.', 'Unknown\nUnknowns', 'Online\nLearn', 'Multi-\nAgent']
severity_short = [10, 9, 9, 8, 7]
timeline_years = [15, 12, 15, 7, 12]

scatter = ax2.scatter(timeline_years, severity_short, s=[s*50 for s in severity_short], 
                     c=severity_short, cmap='Reds', alpha=0.7, edgecolors='black', linewidth=2)

for i, name in enumerate(gap_names):
    ax2.annotate(name, (timeline_years[i], severity_short[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=8, fontweight='bold')

ax2.set_xlabel('Est. Years to Solution', fontsize=11, fontweight='bold')
ax2.set_ylabel('Severity (1-10)', fontsize=11, fontweight='bold')
ax2.set_title('Gap Severity vs. Timeline', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 18)
ax2.set_ylim(6, 11)

# Chart 3: Standards evolution roadmap
ax3 = fig.add_subplot(gs[1, 1])
periods = ['2025-2030', '2030-2035', '2035+']
current = [60, 0, 0]  # Current standards
emerging = [20, 30, 0]  # Emerging standards
future = [0, 20, 40]  # Future standards

x = np.arange(len(periods))
width = 0.6

ax3.bar(x, current, width, label='Current Standards', color='blue', alpha=0.7)
ax3.bar(x, emerging, width, bottom=current, label='Emerging Standards', color='green', alpha=0.7)
ax3.bar(x, future, width, bottom=np.array(current)+np.array(emerging), label='Future Standards', color='gold', alpha=0.7)

ax3.set_ylabel('Relative Contribution (%)', fontsize=11, fontweight='bold')
ax3.set_title('Standards Evolution Roadmap', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(periods)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3, axis='y')

# Chart 4: Recommendations summary
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')

recommendations_text = """
KEY RECOMMENDATIONS

For DEVELOPERS:
✓ Follow standards (baseline), but go beyond
✓ Document gaps explicitly in safety case
✓ Conservative ODD, extensive testing
✓ Monitor research for emerging solutions

For REGULATORS:
✓ Standards = baseline, not complete proof
✓ Demand transparency on limitations
✓ Require safety cases (argumentation)
✓ Allow iterative improvement

For RESEARCHERS:
✓ Focus on critical gaps (validation scale, NN verification)
✓ Collaborate with industry on practical constraints
✓ Engage with standards bodies

BOTTOM LINE:
Standards are necessary tools, not complete solutions.
Many fundamental problems remain open.
Safety is an ongoing process, not a one-time certification.
Critical thinking required - understand limitations.
"""

ax4.text(0.05, 0.95, recommendations_text, transform=ax4.transAxes,
         fontsize=9, verticalalignment='top', family='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

plt.suptitle('Standards Gaps and Open Problems: Complete Summary', fontsize=16, fontweight='bold', y=0.98)

plt.show()

print("\n" + "="*100)
print("WORKSHOP COMPLETE: STANDARDS GAPS AND OPEN PROBLEMS")
print("="*100)
print("\n🎯 Main Lessons:")
print("   1. Standards provide frameworks, not complete solutions")
print("   2. Biggest gaps: Validation scale, NN verification, unknown unknowns")
print("   3. Timeline: 5-15 years for major advances")
print("   4. Use standards, but understand their limitations")
print("   5. Safety is ongoing process, not one-time certification")
print("\n📚 This completes Session 5 and the entire workshop!")
print("   You now have comprehensive understanding of AV safety:")
print("   - Technical: Perception, uncertainty, validation")
print("   - Standards: ISO 26262, 21448, 21434, emerging standards")
print("   - Practical: Deployment challenges, ODD, limitations")
print("="*100)

---

## References and Further Reading

### Standards (Current)
- **ISO 26262:2018** - Road vehicles — Functional safety
- **ISO 21448:2022** - Road vehicles — Safety of the intended functionality (SOTIF)
- **ISO/SAE 21434:2021** - Road vehicles — Cybersecurity engineering

### Standards (Emerging)
- **ISO/IEC TR 5469:2024** - Artificial intelligence — Functional safety and AI systems
- **ISO 34503:2023** - Road vehicles — Test scenarios for automated driving systems
- **UL 4600:2023** - Standard for Safety for the Evaluation of Autonomous Products

### Research Papers on Standards Gaps
- Koopman, P., & Wagner, M. (2017). "Autonomous Vehicle Safety: An Interdisciplinary Challenge" *IEEE ITAS*.
- Burton, S., et al. (2020). "Confidence Arguments for Evidence of Performance in Machine Learning" *SafeComp 2020*.
- Borg, M., et al. (2019). "Safely Entering the Deep: A Review of Verification and Validation for ML" *arXiv:1812.10577*.

### Research Papers on Open Problems
- Kalra, N., & Paddock, S. M. (2016). "Driving to safety" *Transportation Research Part A*.
- Amodei, D., et al. (2016). "Concrete Problems in AI Safety" *arXiv:1606.06565*.
- Huang, X., et al. (2020). "A Survey of Safety and Trustworthiness of Deep Neural Networks" *CSUR*, 53(6).

### Industry Perspectives
- Waymo Safety Report (2024)
- SAE International: "Challenges in Autonomous Vehicle Testing and Validation"
- McKinsey: "The Future of Mobility - Standards and Regulation"

---

*Copyright © 2025 Milin Patel. All Rights Reserved.*

**End of Session 5 and Workshop**

Thank you for participating in this comprehensive workshop on Autonomous Driving: AI, Safety, and Security!